<a href="https://colab.research.google.com/github/marjoriecrowell/Data-mgmt/blob/master/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Final project

Steps in analysis:
* Compile data sources
* Reshape (as needed)

# Datasets and remaining todos

* CPCs - split up city, state, zip
* Facility regulations - a lot of cleaning

## Research Question

My research question is:

How is the number of crisis pregnancy centers in a state affected by abortion provision within the state, health care coverage, and political demographic factors?

I will measure abortion provision by looking at the number of abortion providers within the state and the restrictions on abortion provision within the state.

I will measure health care coverage by looking at the proportion uninsured and whether or not the state has expanded Medicaid. 

My political demographic factors will include voting in the 2016 presidential election and a dataset on poverty within the state.



## Workflow diagram

## Gathering data to address research question

### Dependent variable: number of crisis pregnancy centers in a state

My first dataset will be the number of crisis pregnancy centers within a state, using a dataset from Reproaction.

As a note, on this dataset, I had to remove some notes from the first few columns for the data to be read successfully by pandas, which I did on the back end.

I also had to separate city, state, and zip (I only kept the state) and sum the number of unique CPCs by state, since the number by state is my dependent variable

#### Importing dataset [cpcs]

In [52]:
# I need pandas to re-work the dataset
import pandas as pd

#loading in dataset
cpcs = pd.read_csv("CPCdatabase.csv")

# I want to check the first and last 10 records and the column values
print("1. Getting first 5 records")
print(cpcs.head(5))

print("2. Getting last 5 records")
print(cpcs.tail(5))

print("3. Get column values")
print(cpcs.columns)
print(list(cpcs))

cpcs.to_csv("cpcs.csv")
                       

1. Getting first 5 records
                             Crisis Pregnancy Center  \
0           REAL LIFE PREGNANCY CENTER - ALBERTVILLE   
1                    SAV-A-LIFE OF TALLAPOOSA COUNTY   
2  SAV-A-LIFE FAMILY HOPE CENTER OF COVINGTON COU...   
3                            HOPE WOMEN'S SERVICES     
4                            WOMEN'S RESOURCE CENTER   

                          Street Address     City, State, Zip Code  \
0       104 West Alabama Avenue, Suite F    Albertville, AL, 35950   
1                    11 Lafayette Street  Alexander City, AL 35010   
2  1835 East Three Notch Street, Suite D       Andalusia, AL 36420   
3          1506 Leighton Avenue, Suite A        Anniston, AL 36207   
4               206 South Madison Street          Athens, AL 35612   

               Affiliation                                            Website  \
0                 Care Net                        http://www.reallifecpc.org/   
1                 Care Net                         ht

#### Cleaning dataset

In [53]:
# # There are columns in there I don't need - blanks and social media links 
print("1. Getting rid of last 11 columns")
cpcs = cpcs.drop(cpcs.columns[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]], axis=1)

print("2. Get column values")
print(cpcs.columns)
print(list(cpcs))

print("3. Get rid of 3 blank rows at end")
cpcs.drop(cpcs.tail(3).index,inplace=True)


print("4. Getting last 5 records")
print(cpcs.tail(5))

1. Getting rid of last 11 columns
2. Get column values
Index(['Crisis Pregnancy Center', 'City, State, Zip Code'], dtype='object')
['Crisis Pregnancy Center', 'City, State, Zip Code']
3. Get rid of 3 blank rows at end
4. Getting last 5 records
                    Crisis Pregnancy Center   City, State, Zip Code
2624        HEART TO HEART PREGNANCY CENTER       Laramie, WY 82072
2625                        THE FAMILY TREE      Pinedale, WY 82941
2626  ABBA'S HOUSE PREGNANCY MEDICAL CLINIC     Rivertown, WY 82501
2627                      INSIDE CONNECTION  Rock Springs, WY 82901
2628                LEGACY PREGNANCY CENTER      Sheridan, WY 82801


###### City, state, zip - splitting

In [54]:
print("1. Cleaning up 'city, state, zip' column") 
# I need to rework this dataset - the city, state, and zip are all in one cell
# one attempt from stack overflow: 
cpcs["City, State, Zip Code"]=cpcs["City, State, Zip Code"].str.split(', ').apply(lambda x: x[1])
cpcs["City, State, Zip Code"]=cpcs["City, State, Zip Code"].str.split(' ').apply(lambda x: x[0])

print("2. Checking last 10 records")
print(cpcs.tail())

print("3. Checking first 10 records")
print(cpcs.head())

print("4. Renaming columns")
print(cpcs.columns)
print(list(cpcs.columns))
cpcs.columns = ['CPC name', 'state']

print("5. Re-saving CPCs file")
cpcs.to_csv("cpcs.csv")


1. Cleaning up 'city, state, zip' column
2. Checking last 10 records
                    Crisis Pregnancy Center City, State, Zip Code
2624        HEART TO HEART PREGNANCY CENTER                    WY
2625                        THE FAMILY TREE                    WY
2626  ABBA'S HOUSE PREGNANCY MEDICAL CLINIC                    WY
2627                      INSIDE CONNECTION                    WY
2628                LEGACY PREGNANCY CENTER                    WY
3. Checking first 10 records
                             Crisis Pregnancy Center City, State, Zip Code
0           REAL LIFE PREGNANCY CENTER - ALBERTVILLE                    AL
1                    SAV-A-LIFE OF TALLAPOOSA COUNTY                    AL
2  SAV-A-LIFE FAMILY HOPE CENTER OF COVINGTON COU...                    AL
3                            HOPE WOMEN'S SERVICES                      AL
4                            WOMEN'S RESOURCE CENTER                    AL
4. Renaming columns
Index(['Crisis Pregnancy Center', 'C

###### Creating dataset to capture number CPCs by state

In [55]:
import pandas as pd

cpcs_bystate = cpcs.groupby('state')['CPC name'].count()

print("1. Checking first 5 records")
print(cpcs_bystate.head(5))

print("2. renaming columns")
cpcs_bystate.columns = ['state', 'number_cpcs']


1. Checking first 5 records
state
AK     10
AL     50
AR     45
AZ     48
CA    172
Name: CPC name, dtype: int64
2. renaming columns


### Abortion provision datasets

I am measuring abortion provision within the state by looking at abortion restrictions and at the number of abortion providers within the state.

##### Restrictions, 1: Hospital requirements


https://www.kff.org/womens-health-policy/state-indicator/regulations-on-facilities-and-clinicians-providing-abortions/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

###### Importing dataset [facility_regulations]

In [56]:
import pandas as pd

facility_regulations = pd.read_csv("facility_regulations_KFF.csv", skiprows=2)

# I will check the first and last 5 records, and the column values
print("1. Getting the first 5 records")
print(facility_regulations.head(5))

print("2. Retrieving last 5 records")
print(facility_regulations.tail(5))

print("3. Getting column values")
print(facility_regulations.columns)
                       

1. Getting the first 5 records
        Location  \
0  United States   
1        Alabama   
2         Alaska   
3        Arizona   
4       Arkansas   

  Clinic must meet structural standards comparable to ambulatory surgical centers  \
0                                          17 states                                
1                                                Yes                                
2                                                 No                                
3                                                Yes                                
4                                                Yes                                

  Corridor width and/or procedure room size specified  \
0                                          13 states    
1                                     Corridor width    
2                                                 No    
3                                                 No    
4                                Procedure room size

###### Cleaning dataset

In [57]:

print ("1. Dropping last 16 rows with notes")
facility_regulations.drop(facility_regulations.tail(16).index,inplace=True)

# # 'Location' is the variable I'll need when merging later
# # 'Location' gives the full state name.

# # I want to drop the footnotes column 
# percent_preterm = percent_preterm.drop(percent_preterm.columns[[5]], axis=1) 

print("2. Getting column values")
print(facility_regulations.columns)
print(list(facility_regulations))

print("3. Dropping footnotes column")
facility_regulations = facility_regulations.drop(facility_regulations.columns[[7]], axis=1)

# I want to rename the columns
print("4. Renaming columns")
facility_regulations.columns = ['state', 'ASC requirement', 'corridor/room size', 'dist to hospital', 'transfer agreement w hospital', 'admitting privileges', 'admitting privileges or alt agreement']

print("5. Retrieving last 10 records - to check")
print(facility_regulations.tail(10))

1. Dropping last 16 rows with notes
2. Getting column values
Index(['Location',
       'Clinic must meet structural standards comparable to ambulatory surgical centers',
       'Corridor width and/or procedure room size specified',
       'Maximum distance between clinics and hospital specified',
       'Transfer agreement with hospital in event of complications required',
       'Hospital admitting privileges for clinicians required',
       'Hospital admitting privileges or alternative agreements for clinicians required',
       'Footnotes'],
      dtype='object')
['Location', 'Clinic must meet structural standards comparable to ambulatory surgical centers', 'Corridor width and/or procedure room size specified', 'Maximum distance between clinics and hospital specified', 'Transfer agreement with hospital in event of complications required', 'Hospital admitting privileges for clinicians required', 'Hospital admitting privileges or alternative agreements for clinicians required', 'Footn

#### Restrictions, 2: restrictions on gestational limits

https://www.kff.org/womens-health-policy/state-indicator/gestational-limit-abortions/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

##### Importing dataset [gest_age]

In [58]:
import pandas as pd

print("1. Importing dataset")
gest_age = pd.read_csv("gestational_limits_KFF.csv", skiprows=2)

print("2. Retrieving first 5 records")
print(gest_age.head(5))

print("3. Retrieving last 5 records")
print(gest_age.tail(5))

1. Importing dataset
2. Retrieving first 5 records
        Location  \
0  United States   
1        Alabama   
2         Alaska   
3        Arizona   
4       Arkansas   

  State Prohibits Some Abortions After a Certain Point in Pregnancy  \
0                                             43 Yes                  
1                                                Yes                  
2                                                 No                  
3                                                Yes                  
4                                                Yes                  

  Gestational Limit on Abortions  \
0                            NaN   
1     20 weeks postfertilization   
2                            NaN   
3                Fetal viability   
4     20 weeks postfertilization   

  Abortion Later in Pregnancy Permitted When Pregnancy Threatens Woman's:  \
0                                                NaN                        
1                           Li

##### Cleaning dataset

In [59]:
print("1. Dropping last 17 records")
gest_age.drop(gest_age.tail(17).index,inplace=True)

print("2. Getting column values")
print(gest_age.columns)
print(list(gest_age))

print("3. Dropping footnotes column")
gest_age = gest_age.drop(gest_age.columns[[4]], axis=1) 

print("4. Renaming columns")
gest_age.columns = ['state', 'state prohibits some abortions after certain pt in pregnancy', 'gestational limit', "when is later abortion permitted"]

print("5. Checking last 10")
print(gest_age.tail(10))

1. Dropping last 17 records
2. Getting column values
Index(['Location',
       'State Prohibits Some Abortions After a Certain Point in Pregnancy',
       'Gestational Limit on Abortions',
       'Abortion Later in Pregnancy Permitted When Pregnancy Threatens Woman's:',
       'Footnotes'],
      dtype='object')
['Location', 'State Prohibits Some Abortions After a Certain Point in Pregnancy', 'Gestational Limit on Abortions', "Abortion Later in Pregnancy Permitted When Pregnancy Threatens Woman's:", 'Footnotes']
3. Dropping footnotes column
4. Renaming columns
5. Checking last 10
            state  \
42   South Dakota   
43      Tennessee   
44          Texas   
45           Utah   
46        Vermont   
47       Virginia   
48     Washington   
49  West Virginia   
50      Wisconsin   
51        Wyoming   

   state prohibits some abortions after certain pt in pregnancy  \
42                                                Yes             
43                                             

#### Restrictions, 3: mandatory ultrasounds

https://www.kff.org/womens-health-policy/state-indicator/ultrasound-requirements/

##### Importing dataset [ultrasounds]

In [8]:
import pandas as pd

ultrasounds = pd.read_csv("mandatory_ultrasounds_KFF.csv", skiprows=2)


# I will check the first and last 5 records, and the column values
print("1. Getting the first 5 records")
print(ultrasounds.head(5))

print("2. Retrieving last 5 records")
print(ultrasounds.tail(5))

print("3. Getting column values")
print(ultrasounds.columns)
print(list(ultrasounds))



1. Getting the first 5 records
        Location State has ultrasound requirement?  \
0  United States                            26 Yes   
1        Alabama                               Yes   
2         Alaska                                No   
3        Arizona                               Yes   
4       Arkansas                               Yes   

  Woman must receive information on accessing ultrasound services  \
0                                             14 Yes                
1                                                 No                
2                                                 No                
3                                                 No                
4                                                 No                

  Provision of ultrasound required for abortion and the provider must display and describe image  \
0                                              4 Yes                                               
1                            

##### Cleaning dataset

In [9]:
# There are a lot of rows I don't need in here, so I'll drop some:
print("1. Dropping rows with unnecessary data")
ultrasounds.drop(ultrasounds.tail(16).index,inplace=True)

# These column names are a mess, so I'll rename
print("2. Getting column values")
print(ultrasounds.columns)
print(list(ultrasounds))

# A lot of these columns are specifics about the ultrasound policy
# I only want to look at the presence of a policy, so will drop
print("3. Dropping columns with notes about specifics of policy")
ultrasounds = ultrasounds.drop(ultrasounds.columns[[2, 3, 4, 5, 6, 7]], axis=1)

print("4. Renaming columns")
ultrasounds.columns = ['state', 'ultrasound_required']

print("5. Retrieving last 10 records")
print(ultrasounds.tail(10))

1. Dropping rows with unnecessary data
2. Getting column values
Index(['Location', 'State has ultrasound requirement?',
       'Woman must receive information on accessing ultrasound services',
       'Provision of ultrasound required for abortion and the provider must display and describe image',
       'Provision of ultrasound required for abortion and provider must offer woman opportunity to view image',
       'If ultrasound is performed as part of preparation for abortion, provider must offer woman opportunity to view image',
       'Provision of ultrasound must be offered by provider to woman',
       'Footnotes'],
      dtype='object')
['Location', 'State has ultrasound requirement?', 'Woman must receive information on accessing ultrasound services', 'Provision of ultrasound required for abortion and the provider must display and describe image', 'Provision of ultrasound required for abortion and provider must offer woman opportunity to view image', 'If ultrasound is performed a

#### Restrictions, 4: state policies on abortion
https://www.kff.org/womens-health-policy/state-indicator/state-policies-protecting-or-restricting-legal-status-of-abortion/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

##### Importing dataset [state_policies]

In [60]:
import pandas as pd

state_policies = pd.read_csv("state_policies_Roe_KFF.csv", skiprows=2)

# I will check the first and last 5 records, and the column values
print("1. Getting the first 5 records")
print(state_policies.head(5))

print("2. Retrieving last 5 records")
print(state_policies.tail(5))

print("3. Getting column values")
print(state_policies.columns)
print(list(state_policies))

1. Getting the first 5 records
        Location Pre-Roe Abortion Ban Still State Law  \
0  United States                              Yes - 9   
1        Alabama                                  Yes   
2         Alaska                                   No   
3        Arizona                                  Yes   
4       Arkansas                                  Yes   

  State Law Expressing Intent to Limit Abortion to Maximum Extent Permitted  \
0                                            Yes - 7                          
1                                                 No                          
2                                                 No                          
3                                                 No                          
4                                                Yes                          

  State Law Protecting the Right to Abortion Footnotes  
0                                    Yes - 9       NaN  
1                                    

##### Cleaning dataset

In [61]:
print("1. dropping last 9 rows")
state_policies.drop(state_policies.tail(9).index,inplace=True)

print("2. Dropping columns with notes about specifics of policy")
state_policies = state_policies.drop(state_policies.columns[[4]], axis=1)

print("3. Renaming columns")
state_policies.columns = ['state', 'pre-Roe ban still law', 'law expresses intent to limit abortion as much as permitted', 'law protects right to abortion']

print("4. checking last 10")
print(state_policies.tail(10))

1. dropping last 9 rows
2. Dropping columns with notes about specifics of policy
3. Renaming columns
4. checking last 10
            state pre-Roe ban still law  \
42   South Dakota                    No   
43      Tennessee                    No   
44          Texas                    No   
45           Utah                    No   
46        Vermont                    No   
47       Virginia                    No   
48     Washington                    No   
49  West Virginia                   Yes   
50      Wisconsin                   Yes   
51        Wyoming                    No   

   law expresses intent to limit abortion as much as permitted  \
42                                                 No            
43                                                 No            
44                                                 No            
45                                                 No            
46                                                 No            
47       

#### Number abortions by state

https://data.guttmacher.org/states/table?state=AL+AK+AZ+AR+CA+CO+CT+DE+DC+FL+GA+HI+ID+IL+IN+IA+KS+KY+LA+ME+MD+MA+MI+MN+MS+MO+MT+NE+NV+NH+NJ+NM+NY+NC+ND+OH+OK+OR+PA+RI+SC+SD+TN+TX+UT+VT+VA+WA+WV+WI+WY&topics=66&dataset=data

##### Importing dataset [abortions_bystate]

In [62]:
import pandas as pd

abortions_bystate = pd.read_csv("number_abortions_by_state.csv")


# I will check the first and last 5 records, and the column values
print("1. Getting the first 5 records")
print(abortions_bystate.head(5))

print("2. Retrieving last 5 records")
print(abortions_bystate.tail(5))

print("3. Getting column values")
print(abortions_bystate.columns)
print(list(abortions_bystate))


# # I want to drop the footnotes column
# abortions_bystate = abortions_bystate.drop(abortions_bystate.columns[[2]], axis=1) 

# print("4. Retrieving last 10 records")
# print(abortions_bystate.tail(10))

1. Getting the first 5 records
                                      measure_name  datum state_id  \
0  % of all U.S. abortions, by state of occurrence    0.9       AL   
1  % of all U.S. abortions, by state of occurrence    0.2       AK   
2  % of all U.S. abortions, by state of occurrence    1.4       AZ   
3  % of all U.S. abortions, by state of occurrence    0.5       AR   
4  % of all U.S. abortions, by state of occurrence   17.0       CA   

   state_name  first_year  last_year footnotes  \
0     Alabama        2014        NaN       NaN   
1      Alaska        2014        NaN       NaN   
2     Arizona        2014        NaN       NaN   
3    Arkansas        2014        NaN       NaN   
4  California        2014        NaN       NaN   

                                             sources  
0  Abortion Incidence and Service Availability in...  
1  Abortion Incidence and Service Availability in...  
2  Abortion Incidence and Service Availability in...  
3  Abortion Incidence and S

In [63]:
# Dropping columns
print("1. Dropping columns")
abortions_bystate = abortions_bystate.drop(abortions_bystate.columns[[0, 4, 5, 6, 7]], axis=1) 

print("2. Renaming columns")
abortions_bystate.columns=['number abortions', 'state_id', 'state']

# There are a lot of rows I don't need in here, so I'll drop some:
# Dropping first 51 rows with duplicated data (see notes above)
abortions_bystate.drop(abortions_bystate.head(51).index,inplace=True)


print("3. Checking dataset")
print(abortions_bystate.tail(10))

1. Dropping columns
2. Renaming columns
3. Checking dataset
     number abortions state_id          state
92              550.0       SD   South Dakota
93            13880.0       TN      Tennessee
94            55230.0       TX          Texas
95             2960.0       UT           Utah
96             1400.0       VT        Vermont
97            21080.0       VA       Virginia
98            19230.0       WA     Washington
99             2020.0       WV  West Virginia
100            6050.0       WI      Wisconsin
101             120.0       WY        Wyoming


#### Number abortion providers by state

I will use a dataset from the Guttmacher Institute, found here: https://data.guttmacher.org/states/table?state=AL+AK+AZ+AR+CA+CO+CT+DE+DC+FL+GA+HI+ID+IL+IN+IA+KS+KY+LA+ME+MD+MA+MI+MN+MS+MO+MT+NE+NV+NH+NJ+NM+NY+NC+ND+OH+OK+OR+PA+RI+SC+SD+TN+TX+UT+VT+VA+WA+WV+WI+WY&topics=57&dataset=data which gives data on the number of abortion providers by state in 2014 and the change from 2011-2014 

##### Importing dataset [abortion_providers]

In [64]:
import pandas as pd

abortion_providers = pd.read_csv("number_abortion_providers_by_state.csv")

# I will check the first and last 5 records, and the column values
print("1. Getting the first 5 records")
print(abortion_providers.head(5))

print("2. Retrieving last 5 records")
print(abortion_providers.tail(5))

print("3. Getting column values")
print(abortion_providers.columns)
print(list(abortion_providers))

1. Getting the first 5 records
                     measure_name  datum state_id  state_name  first_year  \
0  % change in the no. of clinics    -17       AL     Alabama        2011   
1  % change in the no. of clinics    -25       AK      Alaska        2011   
2  % change in the no. of clinics    -40       AZ     Arizona        2011   
3  % change in the no. of clinics      0       AR    Arkansas        2011   
4  % change in the no. of clinics     -5       CA  California        2011   

   last_year  footnotes                                            sources  
0     2014.0        NaN  Abortion Incidence and Service Availability in...  
1     2014.0        NaN  Abortion Incidence and Service Availability in...  
2     2014.0        NaN  Abortion Incidence and Service Availability in...  
3     2014.0        NaN  Abortion Incidence and Service Availability in...  
4     2014.0        NaN  Abortion Incidence and Service Availability in...  
2. Retrieving last 5 records
       measure_

##### Cleaning dataset

In [65]:
# Dropping columns
print("1. Dropping columns")
abortion_providers = abortion_providers.drop(abortion_providers.columns[[0, 4, 5, 6, 7]], axis=1) 

print("2. Renaming columns")
abortion_providers.columns=['number providers', 'state_id', 'state']

# This dataset exported with all the different data in different rows by variable 'datum', so I need to remove the first 102
print("3. dropping first 102 rows")
abortion_providers.drop(abortion_providers.head(102).index,inplace=True)

print("4. Checking dataset")
print(abortion_providers.head(51))

1. Dropping columns
2. Renaming columns
3. dropping first 102 rows
4. Checking dataset
     number providers state_id                 state
102                 5       AL               Alabama
103                 3       AK                Alaska
104                 9       AZ               Arizona
105                 3       AR              Arkansas
106               152       CA            California
107                21       CO              Colorado
108                25       CT           Connecticut
109                 3       DE              Delaware
110                 5       DC  District of Columbia
111                71       FL               Florida
112                17       GA               Georgia
113                 4       HI                Hawaii
114                 3       ID                 Idaho
115                24       IL              Illinois
116                 9       IN               Indiana
117                12       IA                  Iowa
118         

#### Mandatory waiting periods by state

##### Importing database [waiting_periods]

In [66]:
import pandas as pd

waiting_periods = pd.read_csv("mandatory_waiting_periods_KFF.csv", skiprows=2)

# I will check the first and last 10 records, and the column values
print("1. Getting the first 5 records")
print(waiting_periods.head(5))

print("2. Retrieving last 5 records")
print(waiting_periods.tail(5))

print("3. Getting column values")
print(waiting_periods.columns)
print(list(waiting_periods))

1. Getting the first 5 records
        Location Mandatory Waiting Period  \
0  United States           27 States: Yes   
1        Alabama                      Yes   
2         Alaska                       No   
3        Arizona                      Yes   
4       Arkansas                      Yes   

  Length of Waiting Period between Counseling and Procedure  \
0                                                NaN          
1                                           48 hours          
2                                                NaN          
3                                           24 hours          
4                                           48 hours          

  In-Person Counseling Necessitates Two Trips to Clinic Footnotes  
0                                     14 States: Yes          NaN  
1                                                 No            1  
2                                                NaN          NaN  
3                                              

##### Cleaning database

In [67]:
# Dropping columns
print("1. Dropping columns")
waiting_periods = waiting_periods.drop(waiting_periods.columns[[4]], axis=1) 

print("2. Renaming columns")
waiting_periods.columns=['state', 'waiting period', 'length waiting period', 'two trips necessary']

# This dataset exported with all the different data in different rows by variable 'datum', so I need to remove the first 102
print("3. dropping last 21 rows")
waiting_periods.drop(waiting_periods.tail(21).index,inplace=True)

print("4. Checking dataset")
print(waiting_periods.tail(10))

1. Dropping columns
2. Renaming columns
3. dropping last 21 rows
4. Checking dataset
            state waiting period length waiting period two trips necessary
42   South Dakota            Yes              72 hours                 Yes
43      Tennessee            Yes              48 hours                 Yes
44          Texas            Yes              24 hours                 Yes
45           Utah            Yes              72 hours                 Yes
46        Vermont             No                   NaN                 NaN
47       Virginia            Yes              24 hours                 Yes
48     Washington             No                   NaN                 NaN
49  West Virginia            Yes              24 hours                  No
50      Wisconsin            Yes              24 hours                 Yes
51        Wyoming             No                   NaN                 NaN


#### State ban on so-called "partial birth" abortions

##### Importing database

In [68]:
import pandas as pd

partial_birth = pd.read_csv("partial_birth_KFF.csv", skiprows=2)

print("1. Previewing first 5 records")
print(partial_birth.head(5))

print("2. Previewing last 5 records")
print(partial_birth.tail(5))

1. Previewing first 5 records
        Location State Has Enacted Ban on Partial Birth Abortion?  \
0  United States                                   20 States: Yes   
1        Alabama                         Enjoined or not Enforced   
2         Alaska                         Enjoined or not Enforced   
3        Arizona                                              Yes   
4       Arkansas                                              Yes   

  Includes an Exception to Protect the Life of the Woman Footnotes  
0                                     18 States: Yes           NaN  
1                                                NaN           NaN  
2                                                NaN           NaN  
3                                                Yes             1  
4                                                Yes             1  
2. Previewing last 5 records
                                             Location  \
61                  1. Policy is currently in effect.  

##### Cleaning database

In [69]:
partial_birth = partial_birth.drop(partial_birth.tail(21).index,inplace=True)

print(partial_birth.tail(5))

AttributeError: ignored

### Health care coverage variables

#### Dataset 1: medicaid expansion by state

https://www.kff.org/health-reform/state-indicator/state-activity-around-expanding-medicaid-under-the-affordable-care-act/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

##### Importing dataset [medicaid_exp]

In [70]:
import pandas as pd

medicaid_exp = pd.read_csv("medicaid_expansion_by_state.csv", skiprows=2)

print("1. Previewing first 5 records")
print(medicaid_exp.head(5))

print("2. Previewing last 5 records")
print(medicaid_exp.tail(5))

1. Previewing first 5 records
        Location      Current Status of Medicaid Expansion Decision Footnotes
0  United States  Adopted-37 states (incl. DC); Not Adopted-14 s...       NaN
1        Alabama                                        Not Adopted       NaN
2         Alaska                                            Adopted         1
3        Arizona                                            Adopted         2
4       Arkansas                                            Adopted         2
2. Previewing last 5 records
                                             Location  \
64  6. On April 18, 2019, the Montana Legislature ...   
65  7. Nebraska voters approved a Medicaid expansi...   
66  8. Utah voters passed a ballot measure in Nove...   
67  9. The Virginia General Assembly approved Medi...   
68  10. Wisconsin covers adults up to 100% FPL in ...   

   Current Status of Medicaid Expansion Decision Footnotes  
64                                           NaN       NaN  
65      

##### Cleaning dataset

In [46]:
# Dropping columns
print("1. Dropping columns")
medicaid_exp = medicaid_exp.drop(medicaid_exp.columns[[2]], axis=1) 

print("2. Renaming columns")
medicaid_exp.columns=['state', 'medicaid expanded']

# This dataset exported with all the different data in different rows by variable 'datum', so I need to remove the first 102
print("3. dropping last 17 rows")
medicaid_exp.drop(medicaid_exp.tail(17).index,inplace=True)

print("4. Checking dataset")
print(medicaid_exp.tail(10))

1. Dropping columns
2. Renaming columns
3. dropping last 17 rows
4. Checking dataset
            state medicaid expanded
42   South Dakota       Not Adopted
43      Tennessee       Not Adopted
44          Texas       Not Adopted
45           Utah           Adopted
46        Vermont           Adopted
47       Virginia           Adopted
48     Washington           Adopted
49  West Virginia           Adopted
50      Wisconsin       Not Adopted
51        Wyoming       Not Adopted


#### Dataset 2: insurance rates by state

In [71]:
import pandas as pd

import pandas as pd

state_insurance = pd.read_csv("state_insurance.csv")

print("1. Previewing first 5 records")
print(state_insurance.head(5))

print("2. Previewing last 5 records")
print(state_insurance.tail(5))

1. Previewing first 5 records
        State Uninsured Rate (2010) Uninsured Rate (2015)  \
0    Alabama                  14.6%                 10.1%   
1     Alaska                  19.9%                 14.9%   
2    Arizona                  16.9%                 10.8%   
3   Arkansas                  17.5%                  9.5%   
4  California                 18.5%                  8.6%   

  Uninsured Rate Change (2010-2015)  \
0                            -4.5%    
1                              -5%    
2                            -6.1%    
3                              -8%    
4                            -9.9%    

   Health Insurance Coverage Change (2010-2015)  \
0                                        215000   
1                                         36000   
2                                        410000   
3                                        234000   
4                                       3826000   

   Employer Health Insurance Coverage (2015)  \
0            

### Political/demographic variables



#### Poverty/insurance rates

In [73]:
import pandas as pd

CPS_poverty = pd.read_csv("CPS_poverty_ins_cov.csv")

# Previewing first and last 5 and the column values
print("1. Printing first 5 columns")
print(CPS_poverty.head(5))

print("2. Printing last 5 columns")
print(CPS_poverty.tail(5))

print("3. Getting column values")
print(CPS_poverty.columns)
print(list(CPS_poverty))

1. Printing first 5 columns
  State Poverty status: total pop In poverty in 2017 Not in poverty in 2017  \
0   NaN                   322,549             39,698                282,851   
1   NaN                     4,806                723                  4,083   
2    AL                       NaN                NaN                    NaN   
3    AK                       717                103                    614   
4    AZ                     6,990                926                  6,064   

  Total insured In poverty and insured  Not in poverty and insured  \
0       294,044                  32,950                    261,094   
1         4,276                     579                      3,698   
2           NaN                     NaN                        NaN   
3           627                      87                        540   
4         6,323                     773                      5,550   

  Total uninsured In poverty and uninsured Not in poverty and uninsured  ...

##### Cleaning dataset

In [0]:
# Dropping columns 10-18

#### Voting behavior in state

##### Importing dataset

In [51]:
import pandas as pd

voting = pd.read_csv("1976-2016-president.csv")

print(voting.tail(10))

      year      state state_po  state_fips  state_cen  state_ic        office  \
3730  2016  Wisconsin       WI          55         35        25  US President   
3731  2016  Wisconsin       WI          55         35        25  US President   
3732  2016    Wyoming       WY          56         83        68  US President   
3733  2016    Wyoming       WY          56         83        68  US President   
3734  2016    Wyoming       WY          56         83        68  US President   
3735  2016    Wyoming       WY          56         83        68  US President   
3736  2016    Wyoming       WY          56         83        68  US President   
3737  2016    Wyoming       WY          56         83        68  US President   
3738  2016    Wyoming       WY          56         83        68  US President   
3739  2016    Wyoming       WY          56         83        68  US President   

                        candidate                 party  writein  \
3730            Moorehead, Monica   work

##### Cleaning dataset

#### State GDP

## Merging datasets

### Abortion provision datasets

In [138]:

cpcs_ab = abortion_providers.merge(cpcs_bystate.to_frame(), right_on='state', left_on='state_id', how="outer")

print("2. Retrieving last 10 records")
print(cpcs_ab.tail(20))

2. Retrieving last 10 records
    number providers state_id           state  CPC name
31                 9       NM      New Mexico        24
32                95       NY        New York       112
33                16       NC  North Carolina        87
34                 1       ND    North Dakota         6
35                12       OH            Ohio       124
36                 3       OK        Oklahoma        43
37                15       OR          Oregon        45
38                20       PA    Pennsylvania       138
39                 3       RI    Rhode Island         4
40                 3       SC  South Carolina        36
41                 1       SD    South Dakota        11
42                 7       TN       Tennessee        54
43                28       TX           Texas       180
44                 2       UT            Utah         6
45                 6       VT         Vermont         9
46                18       VA        Virginia        53
47                

In [139]:
cpcs_abs = cpcs_ab.merge(abortions_bystate, on='state', how="outer")
print("1. Getting the first 10 records")
print(cpcs_abs.head(10))

print("2. Retrieving last 10 records")
print(cpcs_abs.tail(10))

print("3. Getting column values")
print(cpcs_abs.columns)
print(list(cpcs_abs))

1. Getting the first 10 records
   number providers state_id_x                 state  CPC name  \
0                 5         AL               Alabama        50   
1                 3         AK                Alaska        10   
2                 9         AZ               Arizona        48   
3                 3         AR              Arkansas        45   
4               152         CA            California       172   
5                21         CO              Colorado        60   
6                25         CT           Connecticut        20   
7                 3         DE              Delaware         8   
8                 5         DC  District of Columbia         3   
9                71         FL               Florida       157   

   number abortions state_id_y  
0            8020.0         AL  
1            1470.0         AK  
2           12870.0         AZ  
3            4590.0         AR  
4          157350.0         CA  
5           13160.0         CO  
6         

In [140]:
cpcs_ab2 = cpcs_abs.merge(gest_age, on='state', how="outer")
print("1. Getting the first 10 records")
print(cpcs_ab2.head(10))

print("2. Retrieving last 10 records")
print(cpcs_ab2.tail(10))

print("3. Getting column values")
print(cpcs_ab2.columns)
print(list(cpcs_ab2))

1. Getting the first 10 records
   number providers state_id_x                 state  CPC name  \
0               5.0         AL               Alabama      50.0   
1               3.0         AK                Alaska      10.0   
2               9.0         AZ               Arizona      48.0   
3               3.0         AR              Arkansas      45.0   
4             152.0         CA            California     172.0   
5              21.0         CO              Colorado      60.0   
6              25.0         CT           Connecticut      20.0   
7               3.0         DE              Delaware       8.0   
8               5.0         DC  District of Columbia       3.0   
9              71.0         FL               Florida     157.0   

   number abortions state_id_y  \
0            8020.0         AL   
1            1470.0         AK   
2           12870.0         AZ   
3            4590.0         AR   
4          157350.0         CA   
5           13160.0         CO   
6  

In [141]:
cpcs_ab3 = cpcs_ab2.merge(ultrasounds, on='state', how="outer")
print("1. Getting the first 10 records")
print(cpcs_ab3.head(10))

print("2. Retrieving last 10 records")
print(cpcs_ab3.tail(10))

print("3. Getting column values")
print(cpcs_ab3.columns)
print(list(cpcs_ab3))

cpcs_ab3.to_csv("cpcs_abortion.csv")

1. Getting the first 10 records
   number providers state_id_x                 state  CPC name  \
0               5.0         AL               Alabama      50.0   
1               3.0         AK                Alaska      10.0   
2               9.0         AZ               Arizona      48.0   
3               3.0         AR              Arkansas      45.0   
4             152.0         CA            California     172.0   
5              21.0         CO              Colorado      60.0   
6              25.0         CT           Connecticut      20.0   
7               3.0         DE              Delaware       8.0   
8               5.0         DC  District of Columbia       3.0   
9              71.0         FL               Florida     157.0   

   number abortions state_id_y  \
0            8020.0         AL   
1            1470.0         AK   
2           12870.0         AZ   
3            4590.0         AR   
4          157350.0         CA   
5           13160.0         CO   
6  

In [131]:
cpcs_abs3.to_csv("merged_cpcs_abortion.csv")

NameError: ignored

### Health care coverage

### Political/demographic 

## Merging all datasets

## Cleaning up final merged dataset



## Exporting dataset